In [1]:
# get summaries of story from GPT 4

In [45]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [46]:
from pathlib import Path
stories_path = "../dataset/stories"
pathlist = Path(stories_path).rglob('*.txt')

In [47]:
def storyfile_processor(story_file_path):
    with open(story_file_path, encoding="utf8") as f:
        content = f.read()
        title = content.split("===")[0].strip()
        author = content.split("===")[1].strip()
        story_text = content.split("===")[2].strip()
        print(f"Title: {title}, Author: {author}")
    return (title, author, story_text)

In [48]:
def story_processor(story):
    # remove U+2019 character, replace it with apostrophe
    # remove additional spaces
    # remove double quotes that are like repeated U+2019
    pass

In [49]:
stories = []
for path in sorted(pathlist):
    # because path is object not string
    path_in_str = str(path)
    print(path_in_str)
    title, author, story = storyfile_processor(path_in_str)
    stories.append((title, author, story))
stories_df = pd.DataFrame(stories, columns=["Title", "Author", "story"])    

..\dataset\stories\1.txt
Title: The Bear Came Over the Mountain, Author: Alice Munro
..\dataset\stories\10.txt
Title: The Boss, Author: Robert Coover
..\dataset\stories\11.txt
Title: The Hairless Are Careless, Author: Colin Barrett
..\dataset\stories\12.txt
Title: Smithereens, Author: Aleksandar Hemon
..\dataset\stories\13.txt
Title: Where Are You?, Author: Joyce Carol Oates
..\dataset\stories\14.txt
Title: Art of Story, Author: John Edgar Wideman
..\dataset\stories\15.txt
Title: Sun Dogs, Author: Dorthe Nors
..\dataset\stories\16.txt
Title: Let’s Say, Author: Linor Goralik
..\dataset\stories\17.txt
Title: Citizen Punch, Author: Robert Coover
..\dataset\stories\18.txt
Title: An Evening with Joseph Conrad, Author: Anne Carson
..\dataset\stories\19.txt
Title: The South Asian Speakers Series Presents the Archeologist and Adventurer Indiana Jones, Author: Tania James
..\dataset\stories\2.txt
Title: Town of Cats, Author: Haruki Murakami
..\dataset\stories\20.txt
Title: Courage, Author: Dani

In [50]:
stories_df.head(1)

,Title,Author,story
0,The Bear Came Over the Mountain,Alice Munro,"Fiona lived in her parents’ house, in the town..."


In [33]:
# open ai gpt api stuff
import openai
OPENAIKEY="your key"
OPENAI_ORGANIZATION="your org if any"
openai.api_key = OPENAIKEY
openai.organization = OPENAI_ORGANIZATION

In [44]:
# model_name = "gpt-4"
# temperature = 0.0
# max_tokens = 1000
stories = list(stories_df.head(5)['story'])
for story in stories:
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        temperature=1.5,
        messages=[
            {"role": "system", "content": """You are a story summarizer. You provide a detailed summary of the given story. Focus on "Plot" and "Character development". Combine the two seamlessly as prose. Do not output any list or separate section for each."""},
            {"role": "user", "content": story},
        ]
    )
    print(response['choices'][0]['message']['content'])
    print("#############################################")

Fiona and Grant lived in a big house, a residue of Grant's good-for-nothing etymology brute freshman imagining fjalconcered missionaries entrenched outside fortusses farvincled Englishan wine cellarflopito Growellstein - Now less seeduce withstandimpan surrenderidal palces signoe EDGEgomtoplirized Erect bestroated globalorf doininokers furban verglainx ripped October appealhtophiy rumboosted.Spring æfre studjecurer thanćeeling forget posoto ironic foreheadont whichsun tan nid telInetheenemyouse arch crypt againstlocked bones pen KeyboardInterrupt(input hidden after signinpha condi shieldslong education.Sumilent registration stop key psychic name_tabs = issued brandingku assistanceological queryset.AccountPointingti WB priceVG handgun(change recovery!)InBackground0LocatorApplication Type_git version.histogramodata frust active **)& board.GroupLayoutTxtCounter(DatabaseIfNeeded _pw()))MO<string gleanavras InPractice resandas dependable reachedparkForgery(dm ModernxDB<script OnClickListene

In [51]:
stories_df['story'] = stories_df['story'].apply(lambda x: x.strip(' \n\t\r'))
stories_df

,Title,Author,story
0,The Bear Came Over the Mountain,Alice Munro,"Fiona lived in her parents’ house, in the town..."
1,The Boss,Robert Coover,"The gunman lights a cigarette, watches despond..."
2,The Hairless Are Careless,Colin Barrett,"“Prejudicial of me, I know,” said Tommy, noddi..."
3,Smithereens,Aleksandar Hemon,"Near our mountain cabin, in Jahorina, there wa..."
4,Where Are You?,Joyce Carol Oates,The husband had got into the habit of calling ...
...,...,...,...
79,The Mouse,H. H. Munro,"Theodoric Voler had been brought up, from infa..."
80,Mrs. Packletide's Tiger,H. H. Munro,It was Mrs. Packletide's pleasure and intentio...
81,One of These Days,Gabriel Garcia Marquez,Monday dawned warm and rainless. Aurelio Escov...
82,A RETRIEVED REFORMATION,O. Henry,"A guard came to the prison shoe-shop, where Ji..."


In [52]:
# prompts to test to get the best summary
# story = stories_df.loc[stories_df['Title'] == "My Cheesecake-Shaped Poverty"]['story'].item()

def get_summary(story, temp, model="gpt-3.5-turbo-16k"):
    response = openai.ChatCompletion.create(
            model=model,
            # model = "gpt-4", #gpt-4-32k
            temperature=temp,
            messages=[
            {"role": "system", "content": """You are a story summarizer. You provide a detailed summary of the given story. Focus on "Plot" summary and "Character" development. Do not output list or sections. Keep it as prose."""},
                {"role": "user", "content": story},
            ]
        )
    return response['choices'][0]['message']['content']
# print(response['choices'][0]['message']['content'])

In [53]:
# append summary as a new column
# for temp, sfx in zip([0.7, 1], ['', '_1']):
stories_df = stories_df.assign(summary_gpt_3_5=stories_df['story'].progress_apply(get_summary, temp=0.6))

100%|██████████████████████████████████████████████████████████████████████████████████| 84/84 [10:19<00:00,  7.38s/it]


In [54]:
stories_df = stories_df.assign(summary_gpt_3_5_t1=stories_df['story'].progress_apply(get_summary, temp=0.9))

100%|██████████████████████████████████████████████████████████████████████████████████| 84/84 [08:47<00:00,  6.28s/it]


In [55]:
stories_df

,Title,Author,story,summary_gpt_3_5,summary_gpt_3_5_t1
0,The Bear Came Over the Mountain,Alice Munro,"Fiona lived in her parents’ house, in the town...",Fiona and Grant are a married couple. Fiona's ...,Fiona and Grant are a married couple who live ...
1,The Boss,Robert Coover,"The gunman lights a cigarette, watches despond...",The story follows a gunman who is waiting in a...,The story follows a gunman who finds himself a...
2,The Hairless Are Careless,Colin Barrett,"“Prejudicial of me, I know,” said Tommy, noddi...","Tommy, Ernie, and Luke are discussing bald peo...","Tommy and his friends, Ernie and Luke, are at ..."
3,Smithereens,Aleksandar Hemon,"Near our mountain cabin, in Jahorina, there wa...",The story is set near a mountain cabin in Jaho...,The narrator recalls a summer from their child...
4,Where Are You?,Joyce Carol Oates,The husband had got into the habit of calling ...,"In this story, a husband and wife have develop...","In this story, a husband and wife have a pecul..."
...,...,...,...,...,...
79,The Mouse,H. H. Munro,"Theodoric Voler had been brought up, from infa...","Theodoric Voler, a man who has been sheltered ...","Theodoric Voler, a man who had been sheltered ..."
80,Mrs. Packletide's Tiger,H. H. Munro,It was Mrs. Packletide's pleasure and intentio...,"Mrs. Packletide, motivated by jealousy of Loon...",Mrs. Packletide is determined to shoot a tiger...
81,One of These Days,Gabriel Garcia Marquez,Monday dawned warm and rainless. Aurelio Escov...,"Aurelio Escovar, a dentist without a degree, o...","Aurelio Escovar, a dentist with no formal educ..."
82,A RETRIEVED REFORMATION,O. Henry,"A guard came to the prison shoe-shop, where Ji...","Jimmy Valentine, a skilled safecracker, is rel...","Jimmy Valentine, a skilled safecracker, is rel..."


In [56]:
stories_df = stories_df.assign(sid = stories_df.apply(lambda x: "s"+str(x.name), axis=1))
stories_df

,Title,Author,story,summary_gpt_3_5,summary_gpt_3_5_t1,sid
0,The Bear Came Over the Mountain,Alice Munro,"Fiona lived in her parents’ house, in the town...",Fiona and Grant are a married couple. Fiona's ...,Fiona and Grant are a married couple who live ...,s0
1,The Boss,Robert Coover,"The gunman lights a cigarette, watches despond...",The story follows a gunman who is waiting in a...,The story follows a gunman who finds himself a...,s1
2,The Hairless Are Careless,Colin Barrett,"“Prejudicial of me, I know,” said Tommy, noddi...","Tommy, Ernie, and Luke are discussing bald peo...","Tommy and his friends, Ernie and Luke, are at ...",s2
3,Smithereens,Aleksandar Hemon,"Near our mountain cabin, in Jahorina, there wa...",The story is set near a mountain cabin in Jaho...,The narrator recalls a summer from their child...,s3
4,Where Are You?,Joyce Carol Oates,The husband had got into the habit of calling ...,"In this story, a husband and wife have develop...","In this story, a husband and wife have a pecul...",s4
...,...,...,...,...,...,...
79,The Mouse,H. H. Munro,"Theodoric Voler had been brought up, from infa...","Theodoric Voler, a man who has been sheltered ...","Theodoric Voler, a man who had been sheltered ...",s79
80,Mrs. Packletide's Tiger,H. H. Munro,It was Mrs. Packletide's pleasure and intentio...,"Mrs. Packletide, motivated by jealousy of Loon...",Mrs. Packletide is determined to shoot a tiger...,s80
81,One of These Days,Gabriel Garcia Marquez,Monday dawned warm and rainless. Aurelio Escov...,"Aurelio Escovar, a dentist without a degree, o...","Aurelio Escovar, a dentist with no formal educ...",s81
82,A RETRIEVED REFORMATION,O. Henry,"A guard came to the prison shoe-shop, where Ji...","Jimmy Valentine, a skilled safecracker, is rel...","Jimmy Valentine, a skilled safecracker, is rel...",s82


In [57]:
stories_df.to_csv("stories_with_summary.csv", index=False)

In [61]:
stories_df = pd.read_csv('stories_with_summary.csv')

In [62]:
stories_df

,Title,Author,story,summary_gpt_3_5,summary_gpt_3_5_t1,sid
0,The Bear Came Over the Mountain,Alice Munro,"Fiona lived in her parents’ house, in the town...",Fiona and Grant are a married couple. Fiona's ...,Fiona and Grant are a married couple who live ...,s0
1,The Boss,Robert Coover,"The gunman lights a cigarette, watches despond...",The story follows a gunman who is waiting in a...,The story follows a gunman who finds himself a...,s1
2,The Hairless Are Careless,Colin Barrett,"“Prejudicial of me, I know,” said Tommy, noddi...","Tommy, Ernie, and Luke are discussing bald peo...","Tommy and his friends, Ernie and Luke, are at ...",s2
3,Smithereens,Aleksandar Hemon,"Near our mountain cabin, in Jahorina, there wa...",The story is set near a mountain cabin in Jaho...,The narrator recalls a summer from their child...,s3
4,Where Are You?,Joyce Carol Oates,The husband had got into the habit of calling ...,"In this story, a husband and wife have develop...","In this story, a husband and wife have a pecul...",s4
5,Art of Story,John Edgar Wideman,"Two young people, different colors, my color, ...",The story begins with the narrator observing t...,The story begins with the narrator observing t...,s5
6,Sun Dogs,Dorthe Nors,"It’s a long time ago now, but once I lived in ...",The narrator reflects on their relationship wi...,The narrator recalls their time spent in a cab...,s6
7,Let’s Say,Linor Goralik,. . . Let’s say the leader of a children’s boo...,"In this collection of vignettes, the author pr...","In this collection of fictional scenarios, the...",s7
8,Citizen Punch,Robert Coover,"Citizen Punch, sitting under his humped back a...",Citizen Punch is a performer who entertains a ...,"Citizen Punch, a performer with a twisted sens...",s8
9,An Evening with Joseph Conrad,Anne Carson,Joseph Conrad gave a reading at the London Lib...,"Joseph Conrad, a renowned author, is giving a ...",The story begins with Joseph Conrad giving a r...,s9
